In [1]:
from heapq import heappush, heappop
from itertools import combinations, chain # requires python 2.6+
from operator import itemgetter
from collections import defaultdict
from copy import copy
from centroidtracker import CentroidTracker #in github.com/fishMAD, modified from PyImageSearch
import overlay_data_on_position 
from overlay_data_on_position import * 
from overlay_data_on_position import plot_network_communities 
from overlay_data_on_position import plot_data_on_video 
import voronoi_graphs
from utilities import create_colourlist
import imgstore
import networkx as nx
from overlay_data_on_position import plot_data_on_video    
from mpl_toolkits.axes_grid1 import make_axes_locatable
from local_properties import sync_rotation  
import stim_handling as stims 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
revdata = pd.read_pickle('/media/recnodes/Dan_storage/20200203_reversal_data_compiled_full.pickle')

In [9]:
localFBF = pd.read_pickle('/media/recnodes/recnode_2mfish/reversals3m_256_dotbot_20190523_175201.stitched/track/localData_FBF.pickle')

In [10]:
localFBF

,frame,trackid,neighbourDist,localArea,localPackingFraction,localMedianRotation,localRscore,localPolarization,localPDcor,localSpeedScore
0,0.0,209.0,14.560848,617.944939,0.140789,0.936398,1.000000,0.921949,1.000000,0.600000
1,0.0,72.0,21.357923,1076.226951,0.107784,0.929606,0.714286,0.777310,-0.428571,0.857143
2,0.0,117.0,7.955670,455.867319,0.190845,0.563961,0.000000,0.913384,-1.000000,0.200000
3,0.0,200.0,19.116432,1219.739248,0.071327,0.957257,0.400000,0.973098,0.000000,0.000000
4,0.0,96.0,11.409087,245.413737,0.413587,0.791397,0.500000,0.818263,0.085714,0.166667
...,...,...,...,...,...,...,...,...,...,...
116451,12007.0,122.0,15.933123,779.437771,0.148825,-0.982657,0.571429,0.703927,0.107143,0.571429
116452,12007.0,188.0,11.528099,238.040095,0.365485,-0.977617,0.800000,0.594693,0.700000,1.000000
116453,12007.0,159.0,11.137275,542.913755,0.160246,-0.885054,1.000000,0.616988,0.400000,0.200000
116454,12007.0,173.0,12.000447,383.239150,0.227012,-0.984513,0.200000,0.932297,-0.200000,0.000000


In [12]:
localdata = pd.read_pickle('/media/recnodes/Dan_storage/20200407_reversal_localdata_compiled_full.pickle')

In [14]:
localdata[0:5]

,index,frame,trackid,neighbourDist,localArea,localPackingFraction,localMedianRotation,localRscore,localPolarization,localPDcor,...,coh,comment,Time,reversal,firstStim,lastStim,syncTime,trialID,date,groupsize
221723,204571,3518.0,35.0,26.404158,1933.840221,0.052486,0.997051,0.666667,0.943140,0.142857,...,NaN,NaN,87.83685,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221724,204572,3518.0,47.0,19.171949,689.044469,0.126262,0.988901,0.800000,0.954682,-0.300000,...,NaN,NaN,87.83685,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221725,204573,3518.0,40.0,19.906204,1030.857818,0.112528,0.882420,0.714286,0.519981,0.535714,...,NaN,NaN,87.83685,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221726,204574,3518.0,51.0,33.334493,2241.340855,0.038816,0.994526,0.400000,0.778718,0.500000,...,NaN,NaN,87.83685,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221727,204575,3518.0,56.0,38.399963,2904.567629,0.029953,0.997899,0.000000,0.689389,1.000000,...,NaN,NaN,87.83685,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64


In [15]:
localdata.shape

(65851889, 28)

In [16]:
localdata.columns

Index(['index', 'frame', 'trackid', 'neighbourDist', 'localArea',
       'localPackingFraction', 'localMedianRotation', 'localRscore',
       'localPolarization', 'localPDcor', 'localSpeedScore', 'FrameNumber',
       'Timestamp', 'IP', 'nDots', 'dotSize', 'speed', 'dir', 'coh', 'comment',
       'Time', 'reversal', 'firstStim', 'lastStim', 'syncTime', 'trialID',
       'date', 'groupsize'],
      dtype='object')

In [22]:
prestim = pd.read_pickle('/media/recnodes/Dan_storage/20200428_reversal_localdata_compiled_prestim.pickle')

In [23]:
prestim

,index,frame,trackid,neighbourDist,localArea,localPackingFraction,localMedianRotation,localRscore,localPolarization,localPDcor,...,coh,comment,Time,reversal,firstStim,lastStim,syncTime,trialID,date,groupsize
221723,204571,3518.0,35.0,26.404158,1933.840221,0.052486,0.997051,0.666667,0.943140,0.142857,...,NaN,NaN,87.836850,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221724,204572,3518.0,47.0,19.171949,689.044469,0.126262,0.988901,0.800000,0.954682,-0.300000,...,NaN,NaN,87.836850,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221725,204573,3518.0,40.0,19.906204,1030.857818,0.112528,0.882420,0.714286,0.519981,0.535714,...,NaN,NaN,87.836850,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221726,204574,3518.0,51.0,33.334493,2241.340855,0.038816,0.994526,0.400000,0.778718,0.500000,...,NaN,NaN,87.836850,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
221727,204575,3518.0,56.0,38.399963,2904.567629,0.029953,0.997899,0.000000,0.689389,1.000000,...,NaN,NaN,87.836850,0,0,0,-1 days +23:59:50.005082,_20181024_153201_0,,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780600,6441810,7134.0,337.0,16.064722,659.194762,0.153976,0.995774,1.000000,0.988083,-0.028571,...,NaN,NaN,158.340483,0,0,0,-1 days +23:59:59.997008,_20191119_133201_1,,512
2780601,6441811,7134.0,112.0,4.099594,50.120837,1.735805,0.935522,0.200000,0.641227,-0.700000,...,NaN,NaN,158.340483,0,0,0,-1 days +23:59:59.997008,_20191119_133201_1,,512
2780602,6441812,7134.0,118.0,9.425960,302.700916,0.479021,0.978552,1.000000,0.968280,-0.200000,...,NaN,NaN,158.340483,0,0,0,-1 days +23:59:59.997008,_20191119_133201_1,,512
2780603,6441793,7134.0,450.0,20.325102,1012.925054,0.114520,0.957552,0.714286,0.936168,-0.642857,...,NaN,NaN,158.340483,0,0,0,-1 days +23:59:59.997008,_20191119_133201_1,,512


In [25]:
m = prestim.groupby('trialID').mean()
m.groupby(['groupsize']).count()

,index,frame,trackid,neighbourDist,localArea,localPackingFraction,localMedianRotation,localRscore,localPolarization,localPDcor,localSpeedScore,FrameNumber,Timestamp,dir,Time,reversal,firstStim,lastStim
groupsize,,,,,,,,,,,,,,,,,,
64.0,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
128.0,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36
256.0,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
512.0,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [26]:
foo = pd.read_pickle('/media/recnodes/Dan_storage/reversals3m_prestim_local_measures_complete.pickle'  )

In [31]:
foo.groupby(['groupsize']).count()['trialID']

groupsize
128    36
256    22
512    29
64     22
Name: trialID, dtype: int64